In [17]:
allow_train = True
freeze_encoder = False
EPOCHS = 20
BATCH_SIZE = 32
shrink_size = 200

lr = 0.5
weight_deacay = 1e-4
T_max = 5
eta_min = 0.0

In [18]:
import torch
from torch import nn
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingLR

from going_modular import engine_post_train, utils
from going_modular import custom_data_setup_post_train
import helper_functions

In [19]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [20]:
no_transforms = helper_functions.get_augmentation_no_transforms()

In [5]:
train_val_dataloader, class_names = custom_data_setup_post_train.create_train_val_dataloader(transform=no_transforms, batch_size=BATCH_SIZE, device=device, shrink_size=shrink_size)

test_dataloader, class_names = custom_data_setup_post_train.create_test_dataloader(transform=no_transforms, batch_size=BATCH_SIZE, device=device, shrink_size=shrink_size)

exp_dataloader, class_names = custom_data_setup_post_train.create_train_dataloader(transform=no_transforms, batch_size=BATCH_SIZE, device=device, shrink_size=shrink_size)

d:\University Project\University-Project-Code\University-Project-Code\going_modular\custom_data_setup_post_train.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.l

In [21]:
class OneHeadCNN(nn.Module):
    def __init__(self):
        super(OneHeadCNN, self).__init__()
        self.final_head = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.final_head(x)

model = OneHeadCNN().to(device)

In [22]:
# Initialize all weights to 1/3
def init_weights_custom(m):
    if isinstance(m, nn.Linear):
        nn.init.constant_(m.weight, 1/3)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

model.apply(init_weights_custom)

OneHeadCNN(
  (final_head): Linear(in_features=3, out_features=1, bias=True)
)

In [23]:
# Define loss and optimizer

loss_fn = nn.MSELoss().to(device)

optimizer = torch.optim.RAdam(model.parameters(), lr=lr, weight_decay=weight_deacay)
scheduler = CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)

In [24]:
folds_train_results = {
    'loss_train': []
}
folds_val_results = {
    'loss_val': []
}
if allow_train:
    # Set the random seeds
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    # Start the timer
    from timeit import default_timer as timer 
    start_time = timer()

    # Setup training and save the results
    for fold in train_val_dataloader:
        train_results, val_results = engine_post_train.train(model=model,
                            train_dataloader=fold['train_dataloader'],
                            val_dataloader=fold['val_dataloader'],
                            optimizer=optimizer,
                            scheduler=scheduler,
                            loss_fn=loss_fn,
                            epochs=EPOCHS,
                            device=device)
        
        folds_train_results['loss_train'].extend(train_results['loss_train'])

        folds_val_results['loss_val'].extend(val_results['loss_val'])

    # End the timer and print out how long it took
    end_time = timer()
    print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

    utils.save_model(model=model, target_dir='models', model_name='post_train_model.pth')
else:
    model.load_state_dict(torch.load('models/post_train_model.pth', weights_only=True, map_location=device))

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1
loss_train: 2.0187 | loss_val: 1.5594 | acc_val: 0.5156

Epoch: 2
loss_train: 1.7746 | loss_val: 1.8558 | acc_val: 0.5000

Epoch: 3
loss_train: 1.6707 | loss_val: 1.2535 | acc_val: 0.4844

Epoch: 4
loss_train: 1.6685 | loss_val: 1.4349 | acc_val: 0.2500

Epoch: 5
loss_train: 1.7171 | loss_val: 1.4571 | acc_val: 0.2656

Epoch: 6
loss_train: 1.7067 | loss_val: 1.3039 | acc_val: 0.4375

Epoch: 7
loss_train: 1.6575 | loss_val: 1.3272 | acc_val: 0.4062

Epoch: 8
loss_train: 1.6672 | loss_val: 1.3950 | acc_val: 0.3594

Epoch: 9
loss_train: 1.7051 | loss_val: 1.2726 | acc_val: 0.2031

Epoch: 10
loss_train: 1.6750 | loss_val: 1.3362 | acc_val: 0.5156

Epoch: 11
loss_train: 1.6695 | loss_val: 1.3558 | acc_val: 0.4531

Epoch: 12
loss_train: 1.6766 | loss_val: 1.9306 | acc_val: 0.3594

Epoch: 13
loss_train: 1.6420 | loss_val: 1.4015 | acc_val: 0.2500

Epoch: 14
loss_train: 1.6476 | loss_val: 1.8538 | acc_val: 0.3594

Epoch: 15
loss_train: 1.7436 | loss_val: 1.8277 | acc_val: 0.4062

Epoc

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1
loss_train: 1.7620 | loss_val: 1.7921 | acc_val: 0.1562

Epoch: 2
loss_train: 1.7776 | loss_val: 1.8374 | acc_val: 0.2188

Epoch: 3
loss_train: 1.7263 | loss_val: 1.6884 | acc_val: 0.2344

Epoch: 4
loss_train: 1.7134 | loss_val: 1.3832 | acc_val: 0.2656

Epoch: 5
loss_train: 1.6412 | loss_val: 1.3919 | acc_val: 0.1719

Epoch: 6
loss_train: 1.6495 | loss_val: 1.7922 | acc_val: 0.1250

Epoch: 7
loss_train: 1.6716 | loss_val: 1.5039 | acc_val: 0.2656

Epoch: 8
loss_train: 1.6584 | loss_val: 2.0221 | acc_val: 0.1562

Epoch: 9
loss_train: 1.9262 | loss_val: 2.2311 | acc_val: 0.2031

Epoch: 10
loss_train: 1.7751 | loss_val: 1.6295 | acc_val: 0.2656

Epoch: 11
loss_train: 2.0157 | loss_val: 2.6750 | acc_val: 0.1094

Epoch: 12
loss_train: 2.0367 | loss_val: 1.6103 | acc_val: 0.2188

Epoch: 13
loss_train: 2.0922 | loss_val: 2.4235 | acc_val: 0.3594

Epoch: 14
loss_train: 2.2302 | loss_val: 1.5456 | acc_val: 0.1719

Epoch: 15
loss_train: 1.8199 | loss_val: 2.2335 | acc_val: 0.1875

Epoc

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1
loss_train: 2.0308 | loss_val: 2.0309 | acc_val: 0.2656

Epoch: 2
loss_train: 2.1615 | loss_val: 1.4656 | acc_val: 0.2344

Epoch: 3
loss_train: 1.9052 | loss_val: 1.9320 | acc_val: 0.1875

Epoch: 4
loss_train: 2.0045 | loss_val: 1.6526 | acc_val: 0.2031

Epoch: 5
loss_train: 1.7352 | loss_val: 1.8526 | acc_val: 0.2344

Epoch: 6
loss_train: 1.7587 | loss_val: 1.7355 | acc_val: 0.1719

Epoch: 7
loss_train: 1.8898 | loss_val: 1.9692 | acc_val: 0.1094

Epoch: 8
loss_train: 1.8475 | loss_val: 1.4403 | acc_val: 0.2188

Epoch: 9
loss_train: 1.9265 | loss_val: 1.3779 | acc_val: 0.2812

Epoch: 10
loss_train: 1.7514 | loss_val: 2.2655 | acc_val: 0.1250

Epoch: 11
loss_train: 2.6030 | loss_val: 2.8494 | acc_val: 0.0000

Epoch: 12
loss_train: 2.6808 | loss_val: 1.8923 | acc_val: 0.1406

Epoch: 13
loss_train: 1.9944 | loss_val: 1.6423 | acc_val: 0.4062

Epoch: 14
loss_train: 2.2128 | loss_val: 1.5572 | acc_val: 0.1562



KeyboardInterrupt: 

In [10]:
if allow_train:
    helper_functions.plot_loss_curves_post_train(folds_train_results, folds_val_results)

In [11]:
test_results = engine_post_train.test_step(
    model=model,
    dataloader=exp_dataloader,
    loss_fn=loss_fn,
    device=device)

test acc: 0.25
